In [ ]:
# google_play_scraper 라이브러리 설치: 구글 플레이스토어에서 데이터를 쉽게 가져올 수 있게 해줌
!pip install google_play_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.3 MB/s eta 0:00:00


---
"말해보카" 모든 리뷰 데이터 크롤

In [ ]:
# 필요한 패키지와 모듈을 임포트
from google_play_scraper import Sort, reviews_all
import pandas as pd

# 'lang'과 'cont' 변수는 각각 리뷰의 언어와 국가를 설정합니다.
lang = 'ko'  # 리뷰의 언어를 한국어로 설정
cont = 'kr'  # 리뷰를 가져올 국가를 한국으로 설정

# reviews_all 함수를 사용해 앱의 모든 리뷰를 가져옵니다.
result = reviews_all(
    'kr.epopsoft.word',  # 앱의 패키지 이름
    sleep_milliseconds=3000,  # 리뷰 요청 사이의 대기 시간 설정
    lang=lang,  # 리뷰 언어 설정
    country=cont,  # 국가 설정
    sort=Sort.NEWEST  # 리뷰 정렬 방식 설정 (가장 관련성이 높은 순서)
)

# 수집된 리뷰 데이터를 저장할 리스트 초기화
reviews_data = []

# 수집된 모든 리뷰를 리스트에 추가
for item in result:
    review = {
        'reviewId': item['reviewId'],
        'content': item['content'],
        'score': item['score'],
        'thumbsUpCount': item['thumbsUpCount'],
        'reviewCreatedVersion': item['reviewCreatedVersion'],
        'at': item['at'],
        'replyContent': item.get('replyContent', None),
        'repliedAt': item.get('repliedAt', None),
        'appVersion': item.get('appVersion', None)
    }
    reviews_data.append(review)

# 리스트를 pandas DataFrame으로 변환
df_reviews = pd.DataFrame(reviews_data)
print(f'총 리뷰 개수: {len(df_reviews)}')  # 수집된 리뷰의 개수 출력

총 리뷰 개수: 39860


---
2020-01-01 ~ 2024-08-25 기간 데이터만 분류

In [ ]:
from datetime import datetime

# 'at' 컬럼을 datetime 형식으로 변환
df_reviews['at'] = pd.to_datetime(df_reviews['at'])

# 특정 기간 필터링
start_date = '2020-01-01 00:00:00'
end_date = '2024-08-25 23:59:59'

filtered_reviews = df_reviews[(df_reviews['at'] >= start_date) & (df_reviews['at'] <= end_date)]

# 필터링된 리뷰 개수 출력
print(f'필터링된 리뷰 개수: {len(filtered_reviews)}')

필터링된 리뷰 개수: 39836


---
필터 리뷰 데이터 저장

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Google Drive에 파일 저장
filtered_reviews.to_csv('/content/drive/MyDrive/voca_filtered_reviews.csv', encoding='utf-8-sig', index=False, escapechar='\\')

print("파일이 Google Drive에 저장되었습니다.")

Mounted at /content/drive
파일이 Google Drive에 저장되었습니다.


---
한국어가 포함된 리뷰와 한국어가 없는 리뷰 분류

In [ ]:
import re
import pandas as pd

# 한국어 문자가 포함되었는지 확인하는 함수
def contains_korean(text):
    # 정규 표현식을 사용하여 한국어 문자가 포함되어 있는지 확인
    return bool(re.search(r'[ㄱ-ㅎㅏ-ㅣ가-힣]', text))

# 한국어 리뷰와 삭제된 리뷰를 저장할 리스트
korean_reviews_data = []
deleted_reviews_data = []

for item in result:
    content = item.get("content", "")

    # content가 문자열이 아니거나, 한국어가 포함되지 않은 경우
    if not isinstance(content, str) or not contains_korean(content):
        deleted_review = {
            "reviewId": item.get("reviewId"),
            "content": content,
            "score": item.get("score"),
            "thumbsUpCount": item.get("thumbsUpCount"),
            "reviewCreatedVersion": item.get("reviewCreatedVersion"),
            "at": item.get("at"),
            "replyContent": item.get("replyContent"),
            "repliedAt": item.get("repliedAt"),
            "appVersion": item.get("appVersion")
        }
        deleted_reviews_data.append(deleted_review)
    else:
        review = {
            "reviewId": item.get("reviewId"),
            "content": content,
            "score": item.get("score"),
            "thumbsUpCount": item.get("thumbsUpCount"),
            "reviewCreatedVersion": item.get("reviewCreatedVersion"),
            "at": item.get("at"),
            "replyContent": item.get("replyContent"),
            "repliedAt": item.get("repliedAt"),
            "appVersion": item.get("appVersion")
        }
        korean_reviews_data.append(review)

# 리스트를 DataFrame으로 변환
df_korean_reviews = pd.DataFrame(korean_reviews_data)
df_deleted_reviews = pd.DataFrame(deleted_reviews_data)

# 삭제된 리뷰 데이터프레임 출력
df_deleted_reviews

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,34058ceb-1e72-4695-8077-4362a7cd63d3,wonderful education app.,5,0,1.2.301,2024-08-23 07:01:43,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-26 02:19:58,1.2.301
1,dd3fbbef-f444-4ef4-90e6-7400d15c1155,goat,5,0,1.2.301,2024-08-18 03:45:36,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-20 06:48:40,1.2.301
2,ceec84c0-34b3-492a-ae39-48140474925e,good,4,0,1.2.301,2024-08-17 23:05:17,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-20 06:49:10,1.2.301
3,baa03e9a-ce1f-4253-83a9-b806a3e28371,Good,5,0,1.2.301,2024-08-17 15:37:52,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-20 06:49:15,1.2.301
4,a963f3d6-6746-4fce-b8c0-90644c7b67d2,good,5,0,None,2024-08-13 14:20:14,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-14 01:02:51,None
...,...,...,...,...,...,...,...,...,...
1098,f4ebf070-ff21-4ca2-8426-b102c72bf84a,Useful,5,0,1.1.9,2020-05-03 09:13:06,None,NaT,1.1.9
1099,1c998e7e-cfae-4384-b514-7ac71b6e1222,Very good,5,0,1.1.9,2020-05-02 06:16:01,None,NaT,1.1.9
1100,f25db71b-189b-47f1-bb4c-4fb311d953c1,Good,5,0,1.1.9,2020-04-25 17:38:36,None,NaT,1.1.9
1101,b195addd-61df-4158-ab03-7c3cf3dc8a00,٩(*ゝڡ◕๑)۶♥,5,0,1.1.8,2020-03-16 11:30:31,None,NaT,1.1.8


In [ ]:
df_korean_reviews

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,397c165d-7011-4112-8b15-abdbaa4799c8,굿,5,0,1.2.301,2024-08-25 09:41:41,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-26 00:32:58,1.2.301
1,bd2830bb-eae3-4ba9-9487-5d5c46bf2fb8,출퇴근때 하기 좋아요... 다른 언어도 내주면 좋겠다...프랑스어라던가...👀👀,5,0,None,2024-08-25 07:32:53,"안녕하세요, 말해보카입니다.\n\n말해보카 앱에 관심을 가지고 소중한 의견을 남겨주...",2024-08-26 00:49:01,None
2,241a90d7-0cfe-4bf6-83c9-84bc580d3355,기억력부스터 잘쓰고있어요 근데 문장말고 단어만 쭉쭉쭉 말해주는것도 있으면 좋겠어요 ...,5,0,1.2.301,2024-08-25 07:12:07,"안녕하세요, 말해보카입니다.\n\n말해보카 앱에 관심을 가지고 소중한 의견을 남겨주...",2024-08-26 00:49:32,1.2.301
3,705551cf-4b75-4093-b576-70853febd9c4,영어 공부관련해서 찾는중이였는데 루키치 유튜브 ppl보고 오늘 처음 해봤는데 괜찮네...,5,0,None,2024-08-25 06:45:53,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-26 00:33:22,None
4,6b2e7018-9211-4512-9b47-47d39f6d73c8,너무좋아요,5,0,1.2.301,2024-08-25 06:28:34,"안녕하세요, 말해보카입니다.\n\n소중한 리뷰를 작성해 주셔서 감사드리며, 즐겁게 ...",2024-08-26 00:33:27,1.2.301
...,...,...,...,...,...,...,...,...,...
38752,8eb37ef0-6b23-4f84-b35c-32e17272c4b1,화면 구성도 좋고 내용도 탄탄하네요~ 연령에 상관없이 사용하기가 좋아요. 단어도 그...,5,2,1.0.78,2019-12-20 10:30:33,None,NaT,1.0.78
38753,2df8fd8e-8f11-49ee-8d7c-ca6b5f289f81,"캐릭터도 귀엽고, 무엇보다도 말하기를 통해 미션 클리어식으로 진행돼서 재미도 있고 ...",5,9,1.0.78,2019-12-20 09:20:30,None,NaT,1.0.78
38754,f52a2e9e-cf48-4922-bada-c2ae7221d2f7,디자인도 이쁘고 ui가 직관적이라 출퇴근 시간에 게임하듯 잘 이용하고 있어요! 오랜...,5,3,1.0.72,2019-12-20 08:48:46,None,NaT,1.0.72
38755,e110dd5d-02ee-4cf5-b45e-c8ebfa371feb,메뉴가 보기 편하고 직관적이라 좋아요. 혼자있을때 만편하게 게임하듯 영어공부 할 수...,5,5,1.0.78,2019-12-20 08:26:17,None,NaT,1.0.78


In [ ]:
# Google Drive에 파일 저장
df_korean_reviews.to_csv('/content/drive/MyDrive/voca_korean_reviews.csv', encoding='utf-8-sig', index=False, escapechar='\\')
df_deleted_reviews.to_csv('/content/drive/MyDrive/voca_deleted_reviews.csv', encoding='utf-8-sig', index=False, escapechar='\\')

print("파일이 Google Drive에 저장되었습니다.")

파일이 Google Drive에 저장되었습니다.
